In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv')
df['imageId']=df['ImageId_ClassId'].apply(lambda x:x.split('_')[0])
df['ClassId']=df['ImageId_ClassId'].apply(lambda x:x.split('_')[1])
df['hasmask']=~df['EncodedPixels'].isna()
df.head()

,ImageId_ClassId,EncodedPixels,imageId,ClassId,hasmask
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,True
1,0002cc93b.jpg_2,NaN,0002cc93b.jpg,2,False
2,0002cc93b.jpg_3,NaN,0002cc93b.jpg,3,False
3,0002cc93b.jpg_4,NaN,0002cc93b.jpg,4,False
4,00031f466.jpg_1,NaN,00031f466.jpg,1,False


In [3]:
def get_mask(image, cls, encodedPixels):
    shape = image[:,:,0].shape
    mask = np.zeros(shape).T.reshape([-1])
    blocks = encodedPixels.split()
    ng = len(blocks)#//2
    for ig in range(0,ng,2):
        start = int(blocks[ig])
        end = start + int(blocks[ig+1])
        mask[start:end]=cls

    mask = mask.reshape([shape[1],shape[0]]).T
    return mask

import numpy as np
from PIL import Image

for n,g in df.groupby('imageId'):
    filename = n
    image_path = os.path.join('data/images',filename)
    mask_path = os.path.join('data/masks',filename.replace('.jpg','.png'))
    
    image = cv2.imread(image_path)
    mask_shape = image.shape[:-1]
    mask = np.zeros(mask_shape, dtype='uint8')
    for i,elem in g.iterrows():
        classId = int(elem.ClassId)
        if elem.hasmask:
            cls_mask = get_mask(image, classId, elem.EncodedPixels)
        else:
            cls_mask = np.zeros(mask_shape)
        new_values = cls_mask>0
        mask[new_values]=classId

    cv2.imwrite(mask_path, mask)

In [4]:
from mainDeepLabV3 import get_loss,binary_crossentropy_with_logits,weighted_cross_entropy,get_mean_iou,get_weights
from model import Deeplabv3
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from ImageDataGen import ImageDataGenerator as IDG
from keras import backend as K
from glob import glob
import shutil

/share/tfhome/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/share/tfhome/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/share/tfhome/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/share/tfhome/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [5]:
MODEL='model'
model_path = os.path.join('checkpoints', MODEL + '.model')
model_checkpoint = ModelCheckpoint(model_path, monitor='val_mean_iou',mode = 'max', save_best_only=True, verbose=1, period=1)
earlyStopping = EarlyStopping(monitor='val_mean_iou', mode = 'max',patience=6, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_mean_iou', mode = 'max',factor=0.5, patience=4, min_lr=0.0000001, verbose=1)

np.random.seed(0)
img_rows=256
img_cols=1600
num_channels = 3
N_CLASSES=5

In [6]:
from sklearn.model_selection import KFold

mask_path = 'data/masks'
images_path = 'data/images'
files = glob(images_path+'/*')

filenames = [f.split('/')[-1] for f in files]

cv_path ='crossvalid'
if os.path.exists(cv_path):
    shutil.rmtree(cv_path)
os.mkdir(cv_path)
print(cv_path)

kf = KFold(n_splits=10)
X = filenames

for ind,(train_index, test_index) in enumerate(kf.split(X)):
    train_files = [filenames[i] for i in train_index]
    test_files = [filenames[i] for i in test_index]
   
    train_group_images_path = os.path.join(cv_path, 'group_'+str(ind),'train','images/images')
    train_group_masks_path = os.path.join(cv_path, 'group_'+str(ind),'train','masks/masks')
    
    test_group_images_path = os.path.join(cv_path, 'group_'+str(ind),'test','images/images')
    test_group_masks_path = os.path.join(cv_path, 'group_'+str(ind),'test','masks/masks')
    
    os.makedirs(train_group_images_path)
    os.makedirs(train_group_masks_path)
    os.makedirs(test_group_images_path)
    os.makedirs(test_group_masks_path)
    
    for filename in train_files:
        source_file = os.path.join(images_path, filename)
        destination_file = os.path.join(train_group_images_path, filename)
        shutil.copyfile(source_file, destination_file)
        #print(source_file, destination_file)
        
        mask_filename = filename.replace('.jpg','.png')
        source_file = os.path.join(mask_path, mask_filename)
        destination_file = os.path.join(train_group_masks_path, mask_filename)
        shutil.copyfile(source_file, destination_file)
        #print(source_file, destination_file)
        
    for filename in test_files:
        source_file = os.path.join(images_path, filename)
        destination_file = os.path.join(test_group_images_path, filename)
        shutil.copyfile(source_file, destination_file)
        #print(source_file, destination_file)
        
        mask_filename = filename.replace('.jpg','.png')
        source_file = os.path.join(mask_path, mask_filename)
        destination_file = os.path.join(test_group_masks_path, mask_filename)
        shutil.copyfile(source_file, destination_file)
        #print(source_file, destination_file)
    print('{}'.format(ind))

In [13]:
cv_path ='crossvalid'
batchsize=2

In [14]:
data_gen_args = dict(height_shift_range=0.05,
                     width_shift_range=0.05,
                     rotation_range=30,
                     horizontal_flip=True,
                     vertical_flip=True)
image_datagen = IDG(rescale=1.,ismask=False,**data_gen_args)
mask_datagen = IDG(rescale=1.,ismask=True,**data_gen_args, nClasses=N_CLASSES)
image_datagen_val = IDG(rescale=1.,ismask=False)
mask_datagen_val  = IDG(rescale=1.,ismask=True, nClasses=N_CLASSES)
    
seed = 1
k=10
for ind in range(k):    
    train_group_images_path = os.path.join(cv_path, 'group_'+str(ind),'train','images')
    train_group_masks_path = os.path.join(cv_path, 'group_'+str(ind),'train','masks')
    
    test_group_images_path = os.path.join(cv_path, 'group_'+str(ind),'test','images')
    test_group_masks_path = os.path.join(cv_path, 'group_'+str(ind),'test','masks')
    
    image_generator = image_datagen.flow_from_directory(train_group_images_path, target_size=(img_rows,img_cols),
                                                            class_mode=None, seed=seed, batch_size=batchsize)
    mask_generator = mask_datagen.flow_from_directory(train_group_masks_path, target_size=(img_rows,img_cols),
                                                           class_mode=None, seed=seed, batch_size=batchsize, 
                                                      color_mode='grayscale')



    image_generator_val = image_datagen_val.flow_from_directory(test_group_images_path, target_size=(img_rows,img_cols),
                                                            class_mode=None, seed=seed, batch_size=batchsize)
    mask_generator_val = mask_datagen_val.flow_from_directory(test_group_masks_path, target_size=(img_rows,img_cols),
                                                           class_mode=None, seed=seed, batch_size=batchsize, 
                                                              color_mode='grayscale')

    train_generator = zip(image_generator, mask_generator)
    val_generator = zip(image_generator_val, mask_generator_val)
    break

Found 11311 images belonging to 1 classes.
Found 11311 images belonging to 1 classes.
Found 1257 images belonging to 1 classes.
Found 1257 images belonging to 1 classes.


In [9]:
model = Deeplabv3(input_shape=(img_rows, img_cols, num_channels),classes=N_CLASSES) 
weights, w = get_weights(None, N_CLASSES, perPixel=0, enable=False)
weights = K.variable(weights)
model.compile(optimizer='adam', loss=get_loss(weights), metrics=['accuracy', get_mean_iou(N_CLASSES)])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
fcfInput (InputLayer)           (None, 256, 1600, 3) 0                                            
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 128, 800, 32) 864         fcfInput[0][0]                   
__________________________________________________________________________________________________
Conv_BN (BatchNormalization)    (None, 128, 800, 32) 128         Conv[0][0]                       
__________________________________________________________________________________________________
Conv_Relu6 (Activation)         (None, 128, 800, 32) 0           Conv_BN[0][0]                    
__________________________________________________________________________________________________
expanded_c

(?, ?, ?, ?)
(?, 256, 1600, 5)


In [15]:
steps_per_epoch = image_generator.__len__()
validation_steps = image_generator_val.__len__()

In [16]:
train_group_images_path

'crossvalid/group_0/train/images'

In [ ]:
results = model.fit_generator(train_generator,
                               steps_per_epoch=steps_per_epoch,
                               validation_data=val_generator,
                               validation_steps=validation_steps,
                               epochs=50,
                               verbose=1,
                               callbacks=[model_checkpoint, earlyStopping,reduce_lr],
                               shuffle=True)

Epoch 1/50
5150/5656 [==========================>...] - ETA: 2:39 - loss: 0.1484 - acc: 0.9729 - mean_iou: 0.9560

In [ ]:
filename,classId = a[0].split('_')
image_path = os.path.join('train_images',filename)
image = cv2.imread(image_path)

In [ ]:
plt.figure(figsize=(15,5))
plt.imshow(images[0])

In [ ]:
plt.figure(figsize=(15,5))
plt.imshow(masks[0])

In [ ]:
mask=get_mask(image, 255, a[1])

In [ ]:
a1=image
a2=np.tile(np.expand_dims(mask,2),[1,1,3])

In [ ]:
dst = (0.7*a1).astype('uint8') + (a2*0.3).astype('uint8')
plt.figure(figsize=(20,5))
plt.imshow(dst)
plt.axis('off')